## 2 - Augmentation des données (Kilométrage)

**Sources :**
- à compléter
- Base de données initial

**Date export :**  jeudi 19 septembre 2024

## 0 - Import des modules et des données

In [1]:
import pandas as pd

### 0.1 - Dataset 1

In [2]:
filtred_tarifs = pd.read_csv(
    "datasets/tarifs-filtre.csv",
    usecols=[
        "Gare origine", "Gare origine - code UIC",
        "Destination", "Gare destination - code UIC",
        "Prix minimum", "Prix maximum"]
)

In [3]:
filtred_tarifs.head()

,Gare origine,Gare origine - code UIC,Destination,Gare destination - code UIC,Prix minimum,Prix maximum
0,STRASBOURG,87212027,MARNE LA VALLEE CHESSY,87111849,13.0,13.0
1,LAVAL,87478404,MARNE LA VALLEE CHESSY,87111849,9.0,9.0
2,CHAMPAGNE-ARDENNE,87171926,MARNE LA VALLEE CHESSY,87111849,5.0,5.0
3,MARNE LA VALLEE CHESSY,87111849,BOURG ST MAURICE,87741793,14.0,14.0
4,BEZIERS,87781005,MARNE LA VALLEE CHESSY,87111849,15.0,15.0


### 0.2 - Dataset 2

In [4]:
emissions_df = pd.read_csv(
    "datasets/emission-co2-tgv.csv",
    usecols=[
        "Origine", "Origine_uic", 
        "Destination ", "Destination_uic ", 
        "Distance entre les gares"
    ],
    sep=";")

In [5]:
emissions_df.head()

,Origine,Origine_uic,Destination,Destination_uic,Distance entre les gares
0,Paris Montparnasse,87391003,Rennes,87471003,364
1,Paris Montparnasse,87391003,Angers-St-Laud,87484006,303
2,Paris Montparnasse,87391003,Saint-Pierre-des-Corps,87571240,221
3,Paris Est,87113001,Strasbourg,87212027,451
4,Paris Est,87113001,Metz,87192039,352


## 1 - Préparation au merge/concat ;) 

### 1.1 - Vérification de la cohérence des données. 
- Les gares de départ non-redondant -> `UNIQUE`
- Les gares d'arrivée au total -> `COUNT`

In [6]:
print("emissions_df -> unique")
print(f"Destination : {emissions_df["Destination_uic "].nunique()}")
print(f"Origine : {emissions_df["Origine_uic"].nunique()}")

emissions_df -> unique
Destination : 81
Origine : 32


In [7]:
print("emissions_df -> count")
print(f"Destination : {emissions_df["Destination_uic "].count()}")
print(f"Origine : {emissions_df["Origine_uic"].count()}")

emissions_df -> count
Destination : 119
Origine : 119


In [8]:
print("filtred_tarifs")
print(f"Destination : {filtred_tarifs["Gare destination - code UIC"].nunique()}")
print(f"Origine : {filtred_tarifs["Gare origine - code UIC"].nunique()}")

filtred_tarifs
Destination : 169
Origine : 164


In [9]:
print("filtred_tarifs -> count")
print(f"Destination : {filtred_tarifs["Gare destination - code UIC"].count()}")
print(f"Origine : {filtred_tarifs["Gare origine - code UIC"].count()}")

filtred_tarifs -> count
Destination : 1801
Origine : 1801


**Conclusion : Origines et Destinations unique mais une gare de départ peut avoir plusieurs destinations. Il y a un trajet aller et il y a un trajet retours.**

### 1.2 - Compléter le kilométrage déjà présent